In [79]:
from requests_html import AsyncHTMLSession
import json
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
from pyppeteer import launch
import asyncio

In [80]:
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "no-cache",
    "pragma": "no-cache",
    # "sec-ch-ua": '.Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "macOS",
    "upgrade-insecure-requests": "1",
    # "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
}

In [81]:
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.MAC.value]
user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)
user_agent = user_agent_rotator.get_random_user_agent()
user_agent

'Mozilla/5.0 (Macintosh; Intel Mac 0S X) AppleWebKit (KHTML, like Gecko) Chrome Safari'

In [82]:
headers['user-agent'] = user_agent
headers

{'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
 'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'en-US,en;q=0.9',
 'cache-control': 'no-cache',
 'pragma': 'no-cache',
 'sec-ch-ua-mobile': '?0',
 'sec-ch-ua-platform': 'macOS',
 'upgrade-insecure-requests': '1',
 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac 0S X) AppleWebKit (KHTML, like Gecko) Chrome Safari'}

In [168]:
browser = await launch(headless = False)
page = await browser.newPage()

In [169]:
# await page.setExtraHTTPHeaders(headers=headers)
await page.goto('https://www.twitter.com/login')

In [170]:
username = (await page.Jx('//input[@autocomplete="username"]'))[0]
await username.type('milanravenell@gmail.com')
await username.press('Enter')

In [ ]:
phone_number_exists = not (await page.Jx('//span[contains(text(), "Enter your phone number or username")]')) == []
phone_number_exists

False

In [ ]:
text_input = await page.J('input')
password = text_input
await password.type('digitaldashapp')
await password.press('Enter')

In [171]:
await page.goto(f'https://analytics.twitter.com/user/MillyTheYounger/tweets')

In [172]:
tweets_and_replies = (await page.Jx('//a[@id="tweets-and-replies-filter"]'))[0]
await tweets_and_replies.click()

In [193]:
date_picker_button = (await page.Jx('//div[@id="daterange-button"]'))[0]
await date_picker_button.click()

left_calendar = (await page.Jx('//div[@class="calendar left"]'))[0]
prev_button = (await left_calendar.Jx('.//th[@class="prev available"]'))[0]
await prev_button.click()

first_of_the_month = (await left_calendar.Jx('.//td[text()="1"]'))[0]
await first_of_the_month.click()

update_button = (await page.Jx('//button[contains(text(), "Update")]'))[0]
await update_button.click()

In [194]:
tweets = await page.Jx('//li[@class="topn-page tweet-activity-tweet-group"]')
tweets

In [205]:
tweet = tweets[2]
props = await page.evaluate('(element) => element.getAttribute("data-tweet-id")', tweet)
# tweet_id = await prop.getProperties()
print(props)

1338652647264563204


In [202]:
await page.evaluate('el => el.scrollIntoView(false)', tweet)
bounding_box = await tweet.boundingBox()
x = bounding_box['x']
y = bounding_box['y']
width = bounding_box['width']
height = bounding_box['height']
bounding_box

{'x': 12, 'y': 453, 'width': 845.5, 'height': 132}

In [203]:
await page.mouse.click(x+ width - 5, y + 5)

In [206]:
await page.evaluate('el=>el.click()', tweet)

In [198]:
close_button = (await page.Jx('//button[@class="Sheet-close"]'))[0]
await close_button.click()

In [61]:
frame = await (await page.J('iframe')).contentFrame()
impressions_el = (await (await frame.Jx('//div[@class="ep-ImpressionsSection"]'))[0].Jx('.//span[@class="ep-MetricAnimation"]'))[0]
impressions = await frame.evaluate('(e) => e.textContent', impressions_el)
impressions

'45'

In [63]:
engagements_div = (await frame.Jx('//div[@class="ep-EngagementsSection"]'))[0]
engagements_el = (await engagements_div.Jx('.//span[@class="ep-MetricAnimation"]'))[0]
engagements = await frame.evaluate('(e) => e.textContent', engagements_el)
engagements

'3'

In [73]:
def clean_string(text):
    return ''.join(c for c in text if c.isalnum())

In [74]:
record = {}
engagement_rows = await (await engagements_div.J('tbody')).JJ('tr')
for row in engagement_rows:
    engagement_type = (await frame.evaluate(
        '(e) => e.textContent',
        (await row.Jx('.//span[@class="ep-EngagementsTableName"]'))[0]
    ))
    engagement_value = await frame.evaluate(
        '(e) => e.textContent',
        (await row.Jx('.//span[@class="ep-MetricAnimation"]'))[0]
    )

    record[clean_string(engagement_type)] = clean_string(engagement_value)
record

{'Profileclicks': '3'}

In [77]:
close_button = (await page.Jx('//button[@class="Sheet-close"]'))[0]
await close_button.click()